### Informações referentes ao conjunto de dados

    Objetivo dos negócios: analisar o histórico de vídeos de tendências no YouTube
    Com que frequência o dashboard será usado: pelo menos uma vez por dia
    Usuário do dashboard de destino: gerentes de planejamento de anúncios em vídeo
    Conteúdo de dados do dashboard:
        Vídeos de tendências do passado, divididos por dia e categoria
        Vídeos de tendências, divididos por países
        Uma tabela de correspondência entre categorias e países
    Parâmetros segundo os quais os dados devem ser agrupados:
        Data e hora da tendência
        Categoria de vídeo
        País
    Os dados:
        Histórico de tendências — valores absolutos com divisão por dia (dois gráficos: números absolutos e proporção percentual)
        Eventos, discriminados por países — valores relativos (% de eventos)
        A correspondência entre as categorias e os países — valores absolutos (uma tabela)
    Importância: todos os gráficos são igualmente importantes
    A tabela fica armazenada no banco de dados do youtube , criado especialmente para suas necessidades
    Intervalo de atualização de dados: uma vez a cada 24 horas, à meia-noite UTC


Fontes de dados para o dashboard: os engenheiros de dados prometeram criar uma tabela agregada chamada trending_by_time. Segue sua estrutura:    
    
    record_id — chave primária
    region — país / região geográfica
    trending_date — data e hora
    category_title — a categoria de vídeo
    videos_count — o número de vídeos na seção de tendências

### Importando as bibliotecas necessárias e importando o conjunto de dados

In [8]:
import pandas as pd
import datetime as dt

In [16]:
trending = pd.read_csv('trending_by_time.csv')

### Analisando a estrutura dos dados

In [17]:
print(trending.head())

   record_id  region               trending_date    category_title  \
0          1  France  2017-11-14 00:00:00.000000  Autos & Vehicles   
1          2  France  2017-11-15 00:00:00.000000  Autos & Vehicles   
2          3  France  2017-11-16 00:00:00.000000  Autos & Vehicles   
3          4  France  2017-11-17 00:00:00.000000  Autos & Vehicles   
4          5  France  2017-11-18 00:00:00.000000  Autos & Vehicles   

   videos_count  
0             8  
1             2  
2             6  
3             8  
4             4  


    Não preciso realizar a transformação dos cabeçalhos das colunas.

In [18]:
trending.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12343 entries, 0 to 12342
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   record_id       12343 non-null  int64 
 1   region          12343 non-null  object
 2   trending_date   12343 non-null  object
 3   category_title  12343 non-null  object
 4   videos_count    12343 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 482.3+ KB


    São 12343 informações sendo que nenhuma das colunas do DataFrame apresenta valores nulos.
    Preciso transformar o tipo de dados de data de object para datetime. 
    Além disso seria interessante criar uma coluna separando a data em ano, mês e dia, além de hora.

In [39]:
#Tranformando 'trending_date' no formato datetime
trending['trending_date'] = pd.to_datetime(trending['trending_date'], format="%Y-%m-%d %H:%M:%S.%f")

#separando os dados em data, ano, mês, dia e hora
trending['date'] = trending['trending_date'].dt.date
trending['year'] = trending['trending_date'].dt.year
trending['month'] = trending['trending_date'].dt.month
trending['day'] = trending['trending_date'].dt.day
trending['hour'] = trending['trending_date'].dt.hour

In [36]:
print(trending.sample(5))

       record_id         region trending_date    category_title  videos_count  \
11462      11463  United States    2018-05-10    People & Blogs            16   
4926        4927          India    2018-02-01            Sports             8   
4286        4287          India    2018-05-15   News & Politics            76   
1769        1770         France    2018-04-01   News & Politics            26   
10309      10310  United States    2018-01-31  Film & Animation            20   

             date  month  day  year  hour  
11462  2018-05-10      5   10  2018     0  
4926   2018-02-01      2    1  2018     0  
4286   2018-05-15      5   15  2018     0  
1769   2018-04-01      4    1  2018     0  
10309  2018-01-31      1   31  2018     0  


In [40]:
#conferindo a hora máxima e minima para ver se realmente os dados são coletados uma unica vez
print(trending['hour'].max())
print(trending['hour'].min())

0
0


    Já foi me passado que os dados atualizam apenas uma vez por dia a meia noite, e essa informação condiz pois tanto a hora maxima quanto a hora minima é 0.